# PAI CA1 Group 7 EDA

### Importing Libaries

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Loading Dataset

In [2]:
%%time
dataset = pd.read_csv('./dataset/taxi_data.csv')
dataset

CPU times: total: 3.78 s
Wall time: 3.79 s


,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,driver_id,label,driver_name,years_of_exp,gender,car_brand,driver_rating,age,age_of_car
0,0,8.00,143.3,-1.706,-9.271,-1.209,-0.0290,-0.0327,0.0154,2.0,0.00,359,0,Jemmie Cardew,9,Female,Mercedes-Benz,4.7,53,20
1,0,8.00,143.3,-1.417,-9.548,-1.861,-0.0224,0.0050,-0.0258,3.0,0.23,359,0,Jemmie Cardew,9,Female,Mercedes-Benz,4.7,53,20
2,0,8.00,143.3,-0.347,-9.533,-1.205,0.0150,-0.0500,0.0251,9.0,0.23,359,0,Jemmie Cardew,9,Female,Mercedes-Benz,4.7,53,20
3,0,8.00,143.3,-0.601,-9.452,-2.158,0.0045,-0.0117,-0.0041,11.0,0.23,359,0,Jemmie Cardew,9,Female,Mercedes-Benz,4.7,53,20
4,0,8.00,143.3,-0.598,-9.863,-1.673,-0.0004,0.0003,-0.0098,12.0,0.23,359,0,Jemmie Cardew,9,Female,Mercedes-Benz,4.7,53,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824548,1623497638003,3.90,255.0,-0.409,8.514,3.915,0.0153,0.0054,-0.0039,564.0,0.68,232,0,Hilda Hollier,6,Female,Nissan,3.2,53,11
2824549,1623497638003,3.90,257.0,-0.611,8.645,5.009,0.0061,-0.0398,0.0047,565.0,0.54,232,0,Hilda Hollier,6,Female,Nissan,3.2,53,11
2824550,1623497638003,3.90,257.0,-0.321,8.776,3.967,0.0171,0.0390,0.0138,566.0,0.00,232,0,Hilda Hollier,6,Female,Nissan,3.2,53,11
2824551,1623497638003,3.90,257.0,-0.608,8.907,4.300,0.0061,-0.0325,0.0120,567.0,0.00,232,0,Hilda Hollier,6,Female,Nissan,3.2,53,11


In [3]:
dataset.shape

(2824553, 20)

#### Double Checking whether there is any NULL or missing values

In [19]:
print(dataset.isna().sum())
print(dataset.isnull().sum())


bookingID         0
Accuracy          0
Bearing           0
acceleration_x    0
acceleration_y    0
acceleration_z    0
gyro_x            0
gyro_y            0
gyro_z            0
second            0
speed             0
driver_id         0
label             0
driver_name       0
years_of_exp      0
gender            0
car_brand         0
driver_rating     0
age               0
age_of_car        0
dtype: int64
bookingID         0
Accuracy          0
Bearing           0
acceleration_x    0
acceleration_y    0
acceleration_z    0
gyro_x            0
gyro_y            0
gyro_z            0
second            0
speed             0
driver_id         0
label             0
driver_name       0
years_of_exp      0
gender            0
car_brand         0
driver_rating     0
age               0
age_of_car        0
dtype: int64


In [7]:
dataset.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824553 entries, 0 to 2824552
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   bookingID       int64  
 1   Accuracy        float64
 2   Bearing         float64
 3   acceleration_x  float64
 4   acceleration_y  float64
 5   acceleration_z  float64
 6   gyro_x          float64
 7   gyro_y          float64
 8   gyro_z          float64
 9   second          float64
 10  speed           float64
 11  driver_id       int64  
 12  label           int64  
 13  driver_name     object 
 14  years_of_exp    int64  
 15  gender          object 
 16  car_brand       object 
 17  driver_rating   float64
 18  age             int64  
 19  age_of_car      int64  
dtypes: float64(11), int64(6), object(3)
memory usage: 895.3 MB


We can see that our memory load for our pandas dataframe is at 895.3MB, so we are going to shrink the memory load using some techniques

In [26]:
convert_dict = {
    'bookingID': 'int32',
    'Accuracy': 'float16',
    'Bearing': 'float16',
    'acceleration_x': 'float16',
    'acceleration_y': 'float16',
    'acceleration_z': 'float16',
    'gyro_x': 'float16',
    'gyro_y': 'float16',
    'gyro_z': 'float16',
    'second': 'float16',
    'speed': 'float16',
    'driver_id': 'int32',
    'label': 'category',
    'gender': 'category',
    'years_of_exp': 'int16',
    'driver_rating': 'float16',
    'age': 'int16',
    'age_of_car': 'int16'
}

# Convert the columns to the specified types
dataset_new = dataset.astype(convert_dict)
dataset_new.info(memory_usage='deep')

c:\Users\admin\miniconda3\envs\tf\lib\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824553 entries, 0 to 2824552
Data columns (total 20 columns):
 #   Column          Dtype   
---  ------          -----   
 0   bookingID       int32   
 1   Accuracy        float16 
 2   Bearing         float16 
 3   acceleration_x  float16 
 4   acceleration_y  float16 
 5   acceleration_z  float16 
 6   gyro_x          float16 
 7   gyro_y          float16 
 8   gyro_z          float16 
 9   second          float16 
 10  speed           float16 
 11  driver_id       int32   
 12  label           category
 13  driver_name     object  
 14  years_of_exp    int16   
 15  gender          category
 16  car_brand       object  
 17  driver_rating   float16 
 18  age             int16   
 19  age_of_car      int16   
dtypes: category(2), float16(11), int16(3), int32(2), object(2)
memory usage: 464.4 MB


We have successfully shrunked our memory by around 400Mbs